<a href="https://colab.research.google.com/github/sreeyadora/secure_banking_assistant/blob/main/secure_banking_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install rsa


In [11]:
import rsa
import time
import uuid
import json
import base64

# -------------------------------
# 🔐 Cryptographic Utilities
# -------------------------------

def generate_rsa_keys():
    """Generates a public-private RSA key pair."""
    return rsa.newkeys(2048)

def encrypt_message(message_bytes, public_key):
    """Encrypts a message using the recipient's RSA public key."""
    return rsa.encrypt(message_bytes, public_key)

def decrypt_message(ciphertext_bytes, private_key):
    """Decrypts a message using the recipient's RSA private key."""
    return rsa.decrypt(ciphertext_bytes, private_key)

def sign_message(message_bytes, private_key):
    """Generates a digital signature using the sender's private key."""
    return rsa.sign(message_bytes, private_key, 'SHA-256')

def verify_signature(message_bytes, signature, public_key):
    """Verifies a digital signature using the sender's public key."""
    try:
        rsa.verify(message_bytes, signature, public_key)
        return True
    except rsa.VerificationError:
        return False

def generate_nonce():
    """Generates a unique nonce using UUID."""
    return str(uuid.uuid4())

def get_current_timestamp():
    """Returns the current timestamp."""
    return str(time.time())

def is_fresh_timestamp(timestamp_str, allowed_delay_seconds=30):
    """Validates if the timestamp is recent (prevents replay)."""
    try:
        return abs(time.time() - float(timestamp_str)) <= allowed_delay_seconds
    except:
        return False


In [12]:
# -------------------------------
# 🗃️ Simulated In-Memory Client Database
# -------------------------------

client_database = {}

def register_client(client_id, public_key, name, email):
    """Registers a client into the simulated in-memory database."""
    client_database[client_id] = {
        "public_key": public_key,
        "name": name,
        "email": email
    }


In [13]:
# -------------------------------
# 🔑 Generate RSA Key Pairs
# -------------------------------

# Bank (Server) key pair
bank_public_key, bank_private_key = generate_rsa_keys()

# Client key pair
client_public_key, client_private_key = generate_rsa_keys()

# Register the client in the bank's "database"
register_client(
    client_id="customer_001",
    public_key=client_public_key,
    name="Sreeya Dora",
    email="sreeya.dr@gmail.com"
)


In [16]:
# -------------------------------
# 📤 Client-Side: Create Secure Packet
# -------------------------------

# Prepare the message payload
message_payload = {
    "message": "Transfer ₹50,000 to account 321098765432",
    "timestamp": get_current_timestamp(),
    "nonce": generate_nonce()
}

# Convert to bytes
payload_bytes = json.dumps(message_payload).encode()

# Encrypt the payload with bank's public key
encrypted_payload = encrypt_message(payload_bytes, bank_public_key)

# Digitally sign the original message
digital_signature = sign_message(payload_bytes, client_private_key)

# Final secure packet to be sent
secure_packet = {
    "client_id": "customer_001",
    "ciphertext": base64.b64encode(encrypted_payload).decode(),
    "signature": base64.b64encode(digital_signature).decode()
}


In [17]:
# -------------------------------
# 📥 Bank-Side: Process Incoming Secure Packet
# -------------------------------

def process_secure_message(packet):
    """Processes an incoming secure message at the bank."""
    client_id = packet.get("client_id")
    client_record = client_database.get(client_id)

    if not client_record:
        return "❌ Error: Unregistered client"

    # Load client's public key
    sender_public_key = client_record["public_key"]

    # Decode ciphertext and signature
    try:
        encrypted_bytes = base64.b64decode(packet["ciphertext"])
        signature_bytes = base64.b64decode(packet["signature"])
    except:
        return "❌ Error: Invalid encoding in packet"

    # Step 1: Decrypt the message
    try:
        decrypted_bytes = decrypt_message(encrypted_bytes, bank_private_key)
        decrypted_payload = json.loads(decrypted_bytes.decode())
    except:
        return "❌ Error: Failed to decrypt message"

    # Step 2: Replay protection (timestamp check)
    if not is_fresh_timestamp(decrypted_payload["timestamp"]):
        return "⚠️ Replay attack suspected! Timestamp too old or invalid."

    # Step 3: Signature verification
    if not verify_signature(decrypted_bytes, signature_bytes, sender_public_key):
        return "❌ Error: Signature verification failed"

    # Step 4: Success — Message accepted
    return f"✅ Secure Message from {client_record['name']} ({client_id}):\n📝 {decrypted_payload['message']}"


In [18]:
# Simulate the secure communication
response = process_secure_message(secure_packet)
print(response)


✅ Secure Message from Sreeya Dora (customer_001):
📝 Transfer ₹50,000 to account 321098765432
